# Ejercicio Data Quality - Perfilado
## Evaluar la calidad de datos de las ventas de productos

Se quiere hacer una evaluación de calidad de datos sobre las ventas (sales) y pagos (payments). Para ello se requiere hacer un análisis de los siguientes puntos:
- Calidad de los datos
- Selección de clave principal
- Identificación de cardinalidad
- Obtener media, varianza y desviacion Estandar, covarianza, correlacion
- Mejorar la calidad.

**Referencia**: “Estadística Descriptiva con Python y Pandas”: https://coderhook.github.io/Descriptive%20Statistics

- Columnas sales:, orderNumber, orderLineNumber, orderDate, shippedDate, requiredDate, customerNumber, employeeNumber, productCode, status, comments, quantityOrdered, priceEach, sales_amount, origin

- Columnas payments:, customerNumber, checkNumber, paymentDate, amount

In [122]:
import pandas as pd
import numpy as np

In [123]:
# Leer el archivo CSV desde la URL
url = "https://raw.githubusercontent.com/ricardoahumada/DataScienceBasics/refs/heads/main/data/company_sales/sales.csv"
sales_df = pd.read_csv(url)

In [124]:
# Mostrar las primeras filas del DataFrame para entender su estructura
sales_df.columns = [
    'orderNumber', 'orderLineNumber', 'orderDate', 'shippedDate', 'requiredDate', 
    'customerNumber', 'employeeNumber', 'productCode', 'status', 'comments', 
    'quantityOrdered', 'priceEach', 'sales_amount', 'origin'
]
print("Primeras filas del DataFrame:")
print(sales_df.head())

Primeras filas del DataFrame:
   orderNumber  orderLineNumber   orderDate shippedDate requiredDate  \
0        10100                1  0000-00-00  0000-00-00   0000-00-00   
1        10100                2  0000-00-00  0000-00-00   0000-00-00   
2        10100                3  0000-00-00  0000-00-00   0000-00-00   
3        10100                4  0000-00-00  0000-00-00   0000-00-00   
4        10101                1  0000-00-00  0000-00-00   0000-00-00   

   customerNumber  employeeNumber productCode   status  \
0             363            1216    S24_3969  Shipped   
1             363            1216    S18_2248  Shipped   
2             363            1216    S18_1749  Shipped   
3             363            1216    S18_4409  Shipped   
4             128            1504    S18_2795  Shipped   

                 comments  quantityOrdered  priceEach  sales_amount origin  
0                     NaN               49      35.29       1729.21  spain  
1                     NaN         

In [125]:
#se ve que la columna de fechas 'orderDate'
#tiene valores nulos o erroneos. Podemos hacer un sample de 20 filas para asegurarnos
sales_df['orderDate'].sample(20)

444     0000-00-00
2057    0000-00-00
2112    0000-00-00
2174    0000-00-00
2826    0000-00-00
2130    0000-00-00
950     0000-00-00
1760    0000-00-00
1856    0000-00-00
540     0000-00-00
914     0000-00-00
1624    0000-00-00
2244    0000-00-00
2266    0000-00-00
199     0000-00-00
2088    0000-00-00
2059    0000-00-00
2736    0000-00-00
1863    0000-00-00
2818    0000-00-00
Name: orderDate, dtype: object

In [126]:
#verificar con un unique pra ver los valeres únicos
sales_df['orderDate'].unique()
#vemos que existe algún valor que es distinto de 0, pero pocos

array(['0000-00-00', '2038-09-00'], dtype=object)

In [127]:
#por lo que hacemos un value count
sales_df['orderDate'].value_counts()
#vemos que sólo hay tres valores significativos

0000-00-00    2998
2038-09-00       3
Name: orderDate, dtype: int64

In [128]:
#le voy a pedir que identifica las columnas de fecha y valore si están igual de mal y que si es así las borre
# Identificar valores incorrectos en columnas de tipo fecha
threshold = 0.5  # Umbral del 50%
date_columns = ['orderDate', 'shippedDate', 'requiredDate']
columns_to_drop = []

for col in date_columns:
    invalid_dates = sales_df[col].apply(lambda x: pd.to_datetime(x, errors='coerce', format='%Y-%m-%d') is pd.NaT)
    if invalid_dates.mean() > threshold:
        columns_to_drop.append(col)

In [129]:
#elimina las columnas identificadas
sales_df_cleaned = sales_df.drop(columns=columns_to_drop)

In [130]:
# Mostrar las columnas eliminadas
print("\nColumnas eliminadas debido a muchos valores nulos o incorrectos:")
print(columns_to_drop)



Columnas eliminadas debido a muchos valores nulos o incorrectos:
['orderDate', 'shippedDate', 'requiredDate']


In [131]:
# Mostrar información general del DataFrame limpio
print("\nInformación general del DataFrame limpio:")
print(sales_df_cleaned.info())


Información general del DataFrame limpio:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3001 entries, 0 to 3000
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   orderNumber      3001 non-null   int64  
 1   orderLineNumber  3001 non-null   int64  
 2   customerNumber   3001 non-null   int64  
 3   employeeNumber   3001 non-null   int64  
 4   productCode      3001 non-null   object 
 5   status           3001 non-null   object 
 6   comments         759 non-null    object 
 7   quantityOrdered  3001 non-null   int64  
 8   priceEach        3001 non-null   float64
 9   sales_amount     3001 non-null   float64
 10  origin           3001 non-null   object 
dtypes: float64(2), int64(5), object(4)
memory usage: 258.0+ KB
None


In [132]:
# Mostrar estadísticas descriptivas del DataFrame
print("\nEstadísticas descriptivas del DataFrame:")
print(sales_df_cleaned.describe())



Estadísticas descriptivas del DataFrame:
        orderNumber  orderLineNumber  customerNumber  employeeNumber  \
count   3001.000000      3001.000000     3001.000000     3001.000000   
mean   10260.509164         6.424525      259.639120     1317.948684   
std       92.619750         4.196870      118.403435      326.343575   
min    10100.000000         1.000000      103.000000        0.000000   
25%    10181.000000         3.000000      145.000000     1216.000000   
50%    10263.000000         6.000000      240.000000     1370.000000   
75%    10339.000000         9.000000      353.000000     1501.000000   
max    10425.000000        18.000000      496.000000     1702.000000   

       quantityOrdered    priceEach  sales_amount  
count      3001.000000  3001.000000   3001.000000  
mean         35.211929    90.765831   3204.908437  
std           9.828957    36.579368   1631.356967  
min           6.000000    26.550000    481.500000  
25%          27.000000    62.000000   1988.700000

In [133]:
# Verificar la cantidad de valores nulos en cada columna
print("\nCantidad de valores nulos en cada columna:")
print(sales_df_cleaned.isnull().sum())


Cantidad de valores nulos en cada columna:
orderNumber           0
orderLineNumber       0
customerNumber        0
employeeNumber        0
productCode           0
status                0
comments           2242
quantityOrdered       0
priceEach             0
sales_amount          0
origin                0
dtype: int64


In [134]:
# Verificar la cantidad de valores únicos en cada columna
print("\nCantidad de valores únicos en cada columna:")
print(sales_df_cleaned.nunique())


Cantidad de valores únicos en cada columna:
orderNumber         326
orderLineNumber      18
customerNumber       98
employeeNumber       15
productCode         109
status                6
comments             37
quantityOrdered      61
priceEach          1573
sales_amount       2885
origin                2
dtype: int64


In [135]:
# Mostrar la distribución de valores en cada columna categórica
print("\nDistribución de valores en columnas categóricas:")
for column in sales_df_cleaned.select_dtypes(include=['object']).columns:
    print(f"\nDistribución de la columna '{column}':")
    print(sales_df_cleaned[column].value_counts())
    


Distribución de valores en columnas categóricas:

Distribución de la columna 'productCode':
S18_3232    53
S18_2238    29
S18_3136    29
S24_2000    28
S24_4278    28
            ..
S24_3432    25
S24_1628    25
S24_3969    25
S18_4933    24
S24_2887    24
Name: productCode, Length: 109, dtype: int64

Distribución de la columna 'status':
Shipped       2775
Cancelled       79
Resolved        47
On Hold         44
In Process      42
Disputed        14
Name: status, dtype: int64

Distribución de la columna 'comments':
They want to reevaluate their terms agreement with Finance.                                                                                                                                      70
Customer requested that DHL is used for this shipping                                                                                                                                            64
Customer requested that FedEx Ground is used for this shipping                        

In [136]:
sales_df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3001 entries, 0 to 3000
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   orderNumber      3001 non-null   int64  
 1   orderLineNumber  3001 non-null   int64  
 2   customerNumber   3001 non-null   int64  
 3   employeeNumber   3001 non-null   int64  
 4   productCode      3001 non-null   object 
 5   status           3001 non-null   object 
 6   comments         759 non-null    object 
 7   quantityOrdered  3001 non-null   int64  
 8   priceEach        3001 non-null   float64
 9   sales_amount     3001 non-null   float64
 10  origin           3001 non-null   object 
dtypes: float64(2), int64(5), object(4)
memory usage: 258.0+ KB


In [137]:
# Identificar columnas con muchos valores nulos o incorrectos
threshold = 0.5  # Umbral del 50%
columns_to_drop = [col for col in sales_df_cleaned.columns if sales_df_cleaned[col].isnull().mean() > threshold]
columns_to_drop

['comments']

In [138]:
sales_df_cleaned = sales_df_cleaned.drop(columns=['comments'])

In [139]:

sales_df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3001 entries, 0 to 3000
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   orderNumber      3001 non-null   int64  
 1   orderLineNumber  3001 non-null   int64  
 2   customerNumber   3001 non-null   int64  
 3   employeeNumber   3001 non-null   int64  
 4   productCode      3001 non-null   object 
 5   status           3001 non-null   object 
 6   quantityOrdered  3001 non-null   int64  
 7   priceEach        3001 non-null   float64
 8   sales_amount     3001 non-null   float64
 9   origin           3001 non-null   object 
dtypes: float64(2), int64(5), object(3)
memory usage: 234.6+ KB


In [140]:
sales_df_cleaned

,orderNumber,orderLineNumber,customerNumber,employeeNumber,productCode,status,quantityOrdered,priceEach,sales_amount,origin
0,10100,1,363,1216,S24_3969,Shipped,49,35.29,1729.21,spain
1,10100,2,363,1216,S18_2248,Shipped,50,55.09,2754.50,spain
2,10100,3,363,1216,S18_1749,Shipped,30,136.00,4080.00,spain
3,10100,4,363,1216,S18_4409,Shipped,22,75.46,1660.12,spain
4,10101,1,128,1504,S18_2795,Shipped,26,167.06,4343.56,spain
...,...,...,...,...,...,...,...,...,...,...
2996,10425,9,119,1370,S24_2300,In Process,49,127.79,6261.71,spain
2997,10425,10,119,1370,S18_2432,In Process,19,48.62,923.78,spain
2998,10425,11,119,1370,S32_1268,In Process,41,83.79,3435.39,spain
2999,10425,12,119,1370,S10_4962,In Process,38,131.49,4996.62,spain


In [141]:
print(sales_df_cleaned.duplicated().sum())

5


In [142]:
sales_df_cleaned[sales_df_cleaned.duplicated()]

,orderNumber,orderLineNumber,customerNumber,employeeNumber,productCode,status,quantityOrdered,priceEach,sales_amount,origin
28,10104,2,141,1370,S50_1514,Shipped,32,53.31,1705.92,spain
2861,10410,2,357,1612,S18_3136,Shipped,34,84.82,2883.88,spain
2895,10413,6,175,1323,S32_3207,Shipped,24,56.55,1357.20,spain
2945,10419,1,382,1401,S18_1589,Shipped,37,100.80,3729.60,spain
2990,10425,3,119,1370,S18_2238,In Process,28,147.36,4126.08,spain


In [143]:
#busca los duplicados
#Utiliza el método duplicated(keep=False) de pandas para identificar todas las filas duplicadas en el DataFrame sales_df.
#keep=False asegura que todas las ocurrencias de los duplicados sean marcadas como True.

duplicados = sales_df_cleaned[sales_df_cleaned.duplicated(keep=False)]
duplicados

,orderNumber,orderLineNumber,customerNumber,employeeNumber,productCode,status,quantityOrdered,priceEach,sales_amount,origin
27,10104,2,141,1370,S50_1514,Shipped,32,53.31,1705.92,spain
28,10104,2,141,1370,S50_1514,Shipped,32,53.31,1705.92,spain
2860,10410,2,357,1612,S18_3136,Shipped,34,84.82,2883.88,spain
2861,10410,2,357,1612,S18_3136,Shipped,34,84.82,2883.88,spain
2894,10413,6,175,1323,S32_3207,Shipped,24,56.55,1357.20,spain
2895,10413,6,175,1323,S32_3207,Shipped,24,56.55,1357.20,spain
2944,10419,1,382,1401,S18_1589,Shipped,37,100.80,3729.60,spain
2945,10419,1,382,1401,S18_1589,Shipped,37,100.80,3729.60,spain
2989,10425,3,119,1370,S18_2238,In Process,28,147.36,4126.08,spain
2990,10425,3,119,1370,S18_2238,In Process,28,147.36,4126.08,spain


In [144]:
indices_a_eliminar=duplicados.index[::2] # Seleccionar cada segundo índice para eliminar
indices_a_eliminar

Int64Index([27, 2860, 2894, 2944, 2989], dtype='int64')

In [145]:
#elimina los índices a eliminar
sales_df_cleaned = sales_df_cleaned.drop(index=indices_a_eliminar)
sales_df_cleaned

,orderNumber,orderLineNumber,customerNumber,employeeNumber,productCode,status,quantityOrdered,priceEach,sales_amount,origin
0,10100,1,363,1216,S24_3969,Shipped,49,35.29,1729.21,spain
1,10100,2,363,1216,S18_2248,Shipped,50,55.09,2754.50,spain
2,10100,3,363,1216,S18_1749,Shipped,30,136.00,4080.00,spain
3,10100,4,363,1216,S18_4409,Shipped,22,75.46,1660.12,spain
4,10101,1,128,1504,S18_2795,Shipped,26,167.06,4343.56,spain
...,...,...,...,...,...,...,...,...,...,...
2996,10425,9,119,1370,S24_2300,In Process,49,127.79,6261.71,spain
2997,10425,10,119,1370,S18_2432,In Process,19,48.62,923.78,spain
2998,10425,11,119,1370,S32_1268,In Process,41,83.79,3435.39,spain
2999,10425,12,119,1370,S10_4962,In Process,38,131.49,4996.62,spain
